# Hamiltonian Exponentiation - 1st Place Diogo Cruz, Portugal

Below is code and methodology from the winning solution to the Spring 2022 Classiq Coding Competition. 


## Submission Description
The approach is described in the source code file. It uses some methods from the arxiv paper 2001.05983, but significantly modified and built upon for the challenge Hamiltonian and restrictions. Optimizations using pyZX and Qiskit were also employed.

In [2]:
import warnings
warnings.filterwarnings("ignore")

from openfermion import QubitOperator as q
from openfermion import get_sparse_operator

import scipy.sparse.linalg as ssl
import numpy as np
from itertools import permutations

from qiskit.quantum_info import Pauli
# from qiskit.aqua.operators.primitive_ops.pauli_op import PauliOp
# from qiskit.aqua.operators.list_ops.summed_op import SummedOp

from qiskit import QuantumCircuit, QuantumRegister, transpile
# from qiskit.aqua.operators import PauliTrotterEvolution, Suzuki
from qiskit.circuit import Parameter

from qiskit.quantum_info import Operator

from copy import copy, deepcopy

import networkx as nx

#from random import gauss, shuffle
# import pyzx as zx

from networkx.algorithms.approximation import traveling_salesman_problem as tsp

# Local files
# Since the submission only allows one file to be submmited,
# the code of these files has been pasted here.
#from vqe_term_commuting import BronKerbosch # Adapted from https://github.com/teaguetomesh/vqe-term-grouping/blob/master/scaling.py
#from aux_functions import *
#from input_data import input_hamiltonian

# Auxiliary files

This code was originally in separate files. Since the submission only accepts one file submmited, the code has been pasted here.

## input_data.py

In [3]:
input_hamiltonian = """
+ 0.003034656830204855 * IIIYYIIIYY 

+ 0.003034656830204855 * IIIXXIIIYY

+ 0.003034656830204855 * IIIYYIIIXX

+ 0.003034656830204855 * IIIXXIIIXX

- 0.008373361424264817 * YZZZYIIIYY

- 0.008373361424264817 * XZZZXIIIYY

- 0.008373361424264817 * YZZZYIIIXX

- 0.008373361424264817 * XZZZXIIIXX

+ 0.00211113766859809 * YZZYIIIIYY

+ 0.00211113766859809 * XZZXIIIIYY

+ 0.00211113766859809 * YZZYIIIIXX

+ 0.00211113766859809 * XZZXIIIIXX

- 0.00491756976241806 * IIIIIIIIYY

- 0.00491756976241806 * IIIIIIIIXX

+ 0.010540187409026488 * ZIIIIIIIYY

+ 0.010540187409026488 * ZIIIIIIIXX

- 0.0011822832324725804 * IZIIIIIIYY

- 0.0011822832324725804 * IZIIIIIIXX

- 0.0011822832324725804 * IIZIIIIIYY

- 0.0011822832324725804 * IIZIIIIIXX

- 0.00154067008970742 * IIIZIIIIYY

- 0.00154067008970742 * IIIZIIIIXX

+ 0.011733623912074194 * IIIIZIIIYY

+ 0.011733623912074194 * IIIIZIIIXX

+ 0.0027757462269049522 * IIIIIZIIYY

+ 0.0027757462269049522 * IIIIIZIIXX

+ 0.0036202487558837123 * IIIIIIZIYY

+ 0.0036202487558837123 * IIIIIIZIXX

+ 0.0036202487558837123 * IIIIIIIZYY

+ 0.0036202487558837123 * IIIIIIIZXX

+ 0.005996760849734561 * IIYZYIIYZY

+ 0.005996760849734561 * IIXZXIIYZY

+ 0.005996760849734561 * IIYZYIIXZX

+ 0.005996760849734561 * IIXZXIIXZX

+ 0.004802531988356293 * IIYYIIIYZY

+ 0.004802531988356293 * IIXXIIIYZY

+ 0.004802531988356293 * IIYYIIIXZX

+ 0.004802531988356293 * IIXXIIIXZX

- 0.004879740484191497 * YZYIIIIYZY

- 0.004879740484191497 * XZXIIIIYZY

- 0.004879740484191497 * YZYIIIIXZX

- 0.004879740484191497 * XZXIIIIXZX

+ 0.005996760849734561 * IYZZYIYZZY

+ 0.005996760849734561 * IXZZXIYZZY

+ 0.005996760849734561 * IYZZYIXZZX

+ 0.005996760849734561 * IXZZXIXZZX

+ 0.004802531988356293 * IYZYIIYZZY

+ 0.004802531988356293 * IXZXIIYZZY

+ 0.004802531988356293 * IYZYIIXZZX

+ 0.004802531988356293 * IXZXIIXZZX

- 0.004879740484191497 * YYIIIIYZZY

- 0.004879740484191497 * XXIIIIYZZY

- 0.004879740484191497 * YYIIIIXZZX

- 0.004879740484191497 * XXIIIIXZZX

- 0.008373361424264817 * IIIYYYZZZY

- 0.008373361424264817 * IIIXXYZZZY

- 0.008373361424264817 * IIIYYXZZZX

- 0.008373361424264817 * IIIXXXZZZX

+ 0.0307383271773138 * YZZZYYZZZY

+ 0.0307383271773138 * XZZZXYZZZY

+ 0.0307383271773138 * YZZZYXZZZX

+ 0.0307383271773138 * XZZZXXZZZX

- 0.0077644411821215335 * YZZYIYZZZY

- 0.0077644411821215335 * XZZXIYZZZY

- 0.0077644411821215335 * YZZYIXZZZX

- 0.0077644411821215335 * XZZXIXZZZX

- 0.005949019975734247 * IIIIIYZZZY

- 0.005949019975734247 * IIIIIXZZZX

- 0.0351167704024114 * ZIIIIYZZZY

- 0.0351167704024114 * ZIIIIXZZZX

+ 0.0027298828353264134 * IZIIIYZZZY

+ 0.0027298828353264134 * IZIIIXZZZX

+ 0.0027298828353264134 * IIZIIYZZZY

+ 0.0027298828353264134 * IIZIIXZZZX

+ 0.0023679368995844726 * IIIZIYZZZY

+ 0.0023679368995844726 * IIIZIXZZZX

- 0.03305872858775587 * IIIIZYZZZY

- 0.03305872858775587 * IIIIZXZZZX

- 0.0021498576488650843 * IIIIIYIZZY

- 0.0021498576488650843 * IIIIIXIZZX

- 0.0021498576488650843 * IIIIIYZIZY

- 0.0021498576488650843 * IIIIIXZIZX

+ 0.004479074568182561 * IIIIIYZZIY

+ 0.004479074568182561 * IIIIIXZZIX

+ 0.004802531988356293 * IIYZYIIYYI

+ 0.004802531988356293 * IIXZXIIYYI

+ 0.004802531988356293 * IIYZYIIXXI

+ 0.004802531988356293 * IIXZXIIXXI

+ 0.010328819322301622 * IIYYIIIYYI

+ 0.010328819322301622 * IIXXIIIYYI

+ 0.010328819322301622 * IIYYIIIXXI

+ 0.010328819322301622 * IIXXIIIXXI

- 0.003466391848475337 * YZYIIIIYYI

- 0.003466391848475337 * XZXIIIIYYI

- 0.003466391848475337 * YZYIIIIXXI

- 0.003466391848475337 * XZXIIIIXXI

+ 0.004802531988356293 * IYZZYIYZYI

+ 0.004802531988356293 * IXZZXIYZYI

+ 0.004802531988356293 * IYZZYIXZXI

+ 0.004802531988356293 * IXZZXIXZXI

+ 0.010328819322301622 * IYZYIIYZYI

+ 0.010328819322301622 * IXZXIIYZYI

+ 0.010328819322301622 * IYZYIIXZXI

+ 0.010328819322301622 * IXZXIIXZXI

- 0.003466391848475337 * YYIIIIYZYI

- 0.003466391848475337 * XXIIIIYZYI

- 0.003466391848475337 * YYIIIIXZXI

- 0.003466391848475337 * XXIIIIXZXI

+ 0.00211113766859809 * IIIYYYZZYI

+ 0.00211113766859809 * IIIXXYZZYI

+ 0.00211113766859809 * IIIYYXZZXI

+ 0.00211113766859809 * IIIXXXZZXI

- 0.0077644411821215335 * YZZZYYZZYI

- 0.0077644411821215335 * XZZZXYZZYI

- 0.0077644411821215335 * YZZZYXZZXI

- 0.0077644411821215335 * XZZZXXZZXI

+ 0.006575744899182541 * YZZYIYZZYI

+ 0.006575744899182541 * XZZXIYZZYI

+ 0.006575744899182541 * YZZYIXZZXI

+ 0.006575744899182541 * XZZXIXZZXI

+ 0.023557442395837284 * IIIIIYZZYI

+ 0.023557442395837284 * IIIIIXZZXI

+ 0.010889407716094479 * ZIIIIYZZYI

+ 0.010889407716094479 * ZIIIIXZZXI

- 0.0003518893528389501 * IZIIIYZZYI

- 0.0003518893528389501 * IZIIIXZZXI

- 0.0003518893528389501 * IIZIIYZZYI

- 0.0003518893528389501 * IIZIIXZZXI

- 0.00901204279263803 * IIIZIYZZYI

- 0.00901204279263803 * IIIZIXZZXI

+ 0.0127339139792953 * IIIIZYZZYI

+ 0.0127339139792953 * IIIIZXZZXI

- 0.003818281201314288 * IIIIIYIZYI

- 0.003818281201314288 * IIIIIXIZXI

- 0.003818281201314288 * IIIIIYZIYI

- 0.003818281201314288 * IIIIIXZIXI

+ 0.004217284878422759 * IYYIIIYYII

+ 0.004217284878422759 * IXXIIIYYII

+ 0.004217284878422759 * IYYIIIXXII

+ 0.004217284878422759 * IXXIIIXXII

- 0.004879740484191498 * IIYZYYZYII

- 0.004879740484191498 * IIXZXYZYII

- 0.004879740484191498 * IIYZYXZXII

- 0.004879740484191498 * IIXZXXZXII

- 0.003466391848475337 * IIYYIYZYII

- 0.003466391848475337 * IIXXIYZYII

- 0.003466391848475337 * IIYYIXZXII

- 0.003466391848475337 * IIXXIXZXII

+ 0.004868302545087521 * YZYIIYZYII

+ 0.004868302545087521 * XZXIIYZYII

+ 0.004868302545087521 * YZYIIXZXII

+ 0.004868302545087521 * XZXIIXZXII

- 0.004879740484191498 * IYZZYYYIII

- 0.004879740484191498 * IXZZXYYIII

- 0.004879740484191498 * IYZZYXXIII

- 0.004879740484191498 * IXZZXXXIII

- 0.003466391848475337 * IYZYIYYIII

- 0.003466391848475337 * IXZXIYYIII

- 0.003466391848475337 * IYZYIXXIII

- 0.003466391848475337 * IXZXIXXIII

+ 0.004868302545087521 * YYIIIYYIII

+ 0.004868302545087521 * XXIIIYYIII

+ 0.004868302545087521 * YYIIIXXIII

+ 0.004868302545087521 * XXIIIXXIII

- 0.004917569762418068 * IIIYYIIIII

- 0.004917569762418068 * IIIXXIIIII

+ 0.0027757462269049522 * ZIIYYIIIII

+ 0.0027757462269049522 * ZIIXXIIIII

+ 0.0036202487558837123 * IZIYYIIIII

+ 0.0036202487558837123 * IZIXXIIIII

+ 0.0036202487558837123 * IIZYYIIIII

+ 0.0036202487558837123 * IIZXXIIIII

- 0.005949019975734285 * YZZZYIIIII

- 0.005949019975734285 * XZZZXIIIII

- 0.0021498576488650843 * YIZZYIIIII

- 0.0021498576488650843 * XIZZXIIIII

- 0.0021498576488650843 * YZIZYIIIII

- 0.0021498576488650843 * XZIZXIIIII

+ 0.004479074568182561 * YZZIYIIIII

+ 0.004479074568182561 * XZZIXIIIII

+ 0.02355744239583729 * YZZYIIIIII

+ 0.02355744239583729 * XZZXIIIIII

- 0.003818281201314288 * YIZYIIIIII

- 0.003818281201314288 * XIZXIIIIII

- 0.003818281201314288 * YZIYIIIIII

- 0.003818281201314288 * XZIXIIIIII

+ 1.0709274663656798 * IIIIIIIIII

- 0.5772920990654371 * ZIIIIIIIII

- 0.4244817531727133 * IZIIIIIIII

+ 0.06245512523136934 * ZZIIIIIIII

- 0.4244817531727134 * IIZIIIIIII

+ 0.06245512523136934 * ZIZIIIIIII

+ 0.06558452315458405 * IZZIIIIIII

- 0.3899177647415215 * IIIZIIIIII

+ 0.053929860773588405 * ZIIZIIIIII

+ 0.06022550139954594 * IZIZIIIIII

+ 0.06022550139954594 * IIZZIIIIII

+ 0.004360552555030484 * YZZYZIIIII

+ 0.004360552555030484 * XZZXZIIIII

- 0.30101532158947975 * IIIIZIIIII

+ 0.08360121967246183 * ZIIIZIIIII

+ 0.06278876343471208 * IZIIZIIIII

+ 0.06278876343471208 * IIZIZIIIII

+ 0.053621410722614865 * IIIZZIIIII

+ 0.010540187409026488 * IIIYYZIIII

+ 0.010540187409026488 * IIIXXZIIII

- 0.0351167704024114 * YZZZYZIIII

- 0.0351167704024114 * XZZZXZIIII

+ 0.010889407716094479 * YZZYIZIIII

+ 0.010889407716094479 * XZZXIZIIII

- 0.5772920990654372 * IIIIIZIIII

+ 0.11409163501020725 * ZIIIIZIIII

+ 0.06732342777645686 * IZIIIZIIII

+ 0.06732342777645686 * IIZIIZIIII

+ 0.060505605672770954 * IIIZIZIIII

+ 0.11433954684977561 * IIIIZZIIII

- 0.0011822832324725804 * IIIYYIZIII

- 0.0011822832324725804 * IIIXXIZIII

+ 0.0027298828353264134 * YZZZYIZIII

+ 0.0027298828353264134 * XZZZXIZIII

- 0.0003518893528389501 * YZZYIIZIII

- 0.0003518893528389501 * XZZXIIZIII

- 0.42448175317271336 * IIIIIIZIII

+ 0.06732342777645686 * ZIIIIIZIII

+ 0.0782363777898523 * IZIIIIZIII

+ 0.06980180803300681 * IIZIIIZIII

+ 0.07055432072184756 * IIIZIIZIII

+ 0.06878552428444665 * IIIIZIZIII

+ 0.06245512523136934 * IIIIIZZIII

- 0.0011822832324725804 * IIIYYIIZII

- 0.0011822832324725804 * IIIXXIIZII

+ 0.0027298828353264134 * YZZZYIIZII

+ 0.0027298828353264134 * XZZZXIIZII

- 0.0003518893528389501 * YZZYIIIZII

- 0.0003518893528389501 * XZZXIIIZII

- 0.42448175317271336 * IIIIIIIZII

+ 0.06732342777645686 * ZIIIIIIZII

+ 0.06980180803300681 * IZIIIIIZII

+ 0.0782363777898523 * IIZIIIIZII

+ 0.07055432072184756 * IIIZIIIZII

+ 0.06878552428444665 * IIIIZIIZII

+ 0.06245512523136934 * IIIIIZIZII

+ 0.06558452315458405 * IIIIIIZZII

- 0.00154067008970742 * IIIYYIIIZI

- 0.00154067008970742 * IIIXXIIIZI

+ 0.0023679368995844726 * YZZZYIIIZI

+ 0.0023679368995844726 * XZZZXIIIZI

- 0.00901204279263803 * YZZYIIIIZI

- 0.00901204279263803 * XZZXIIIIZI

- 0.38991776474152134 * IIIIIIIIZI

+ 0.060505605672770954 * ZIIIIIIIZI

+ 0.07055432072184756 * IZIIIIIIZI

+ 0.07055432072184756 * IIZIIIIIZI

+ 0.08470391802239534 * IIIZIIIIZI

+ 0.05665606755281972 * IIIIZIIIZI

+ 0.053929860773588405 * IIIIIZIIZI

+ 0.06022550139954594 * IIIIIIZIZI

+ 0.06022550139954594 * IIIIIIIZZI

+ 0.004360552555030484 * IIIIIYZZYZ

+ 0.004360552555030484 * IIIIIXZZXZ

+ 0.011733623912074194 * IIIYYIIIIZ

+ 0.011733623912074194 * IIIXXIIIIZ

- 0.03305872858775587 * YZZZYIIIIZ

- 0.03305872858775587 * XZZZXIIIIZ

+ 0.0127339139792953 * YZZYIIIIIZ

+ 0.0127339139792953 * XZZXIIIIIZ

- 0.30101532158947975 * IIIIIIIIIZ

+ 0.11433954684977561 * ZIIIIIIIIZ

+ 0.06878552428444665 * IZIIIIIIIZ

+ 0.06878552428444665 * IIZIIIIIIZ

+ 0.05665606755281972 * IIIZIIIIIZ

+ 0.12357087224898464 * IIIIZIIIIZ

+ 0.08360121967246183 * IIIIIZIIIZ

+ 0.06278876343471208 * IIIIIIZIIZ

+ 0.06278876343471208 * IIIIIIIZIZ

+ 0.053621410722614865 * IIIIIIIIZZ
"""

## aux_functions.py

In [4]:
import numpy as np
from qiskit.quantum_info import Pauli
# from qiskit.aqua.operators.primitive_ops.pauli_op import PauliOp
# from qiskit.aqua.operators.list_ops.summed_op import SummedOp

def get_openfermion_str(pauli_term, reverse=False):
    
    cirq_term = []
    
    for i, op in enumerate(list(pauli_term)):
        if op == 'I':
            continue
        
        cirq_term.append(op + str(9-i if not reverse else i))
        
    new_pauli_term = ' '.join(cirq_term)
    
    return new_pauli_term

def ops_commute(op1, op2):
    sign = 1
    for pauli_1, pauli_2 in zip(list(op1), list(op2)):
        if pauli_1=='I' or pauli_2=='I' or pauli_1==pauli_2:
            continue
        sign *= -1
    
    return True if sign==1 else False

def ops_do_not_overlap(op_1, op_2):
    qbs = []
    for i, (p1, p2) in enumerate(zip(list(op_1), list(op_2))):
        if p1!='I' and p2!='I':
            return False
    return True

def match_weight(op_1, op_2):
    weight = 0.
    for p1, p2 in zip(list(op_1), list(op_2)):
        if p1=='I' and p2=='I':
            continue
        elif p1=='I' or p2=='I':
            weight += 1.
        elif p1 == p2:
            weight += 0.
        else:
            weight += 2.
    return weight

# From https://stackoverflow.com/a/10824420
def flatten(container):
    for i in container:
        if isinstance(i, (list,tuple)):
            yield from flatten(i)
        else:
            yield i

def convert_to_qiskit(H_corr, n_qubits):
    """
    Converts QubitHamiltonian from Cirq/Openfermion into SummedOp from 
    Qiskit.

    Parameters
    ----------
    H_corr : QubitOperator
        Spin Hamiltonian.
    n_qubits : int
        Number of qubits.

    Returns
    -------
    SummedOp
        `H_corr` Hamiltonian in SummedOp form. Note that Qiskit is big 
        endian, so the first qubit has the highest index, while the last 
        qubit has the first index, unlike OpenFermion.
    """

    oplist = []
    id = ['I'] * n_qubits
    for op, coeff in H_corr.terms.items():
        op_str = id.copy()
        for ind, pauli_op in list(op):
            op_str[ind] = pauli_op
        op_str = ''.join(reversed(op_str))
        pauli = PauliOp(Pauli(op_str), coeff=coeff)
        oplist.append(pauli)    

    H = SummedOp(oplist)

    return H
    
def nthperm(l, n):
    l = list(l)

    indices = []
    for i in range(1, 1+len(l)):
        indices.append(n % i)
        n //= i
    indices.reverse()

    perm = []
    for index in indices:
        # Using pop is kind of inefficient. We could probably avoid it.
        perm.append(l.pop(index))
    return tuple(perm)

def compare_with_exact(U_exp, U_exact, global_phase = -1.0709274663656798, circuit_phase = np.pi/2, get_result=False):
    # Note: the default circuit phase is specific to our submitted solution.
    total_phase = global_phase + circuit_phase
    diff_matrix = np.exp(1j*total_phase) * U_exp - U_exact
    error = np.linalg.norm(diff_matrix, ord=2)
    
    total_U = np.exp(-1j*total_phase) * U_exp.T.conj() @ U_exact
    corr_phase = np.mean(np.angle(np.diag(total_U)))
    corr_error = np.linalg.norm(np.exp(-1j*corr_phase) * total_U - np.eye(2**10), ord=2)
    print("Error: {}, Corr: {}, Phase: {}.".format(error, corr_error, corr_phase))
    
    if get_result:
        return error, corr_error

## vqe_term_commuting.py

In [20]:
# -*- coding: utf-8 -*- 
"""
Given a particular qubit Hamiltonian, measuring the expected energy of any
given quantum state will depend only on the individual terms of that 
Hamiltonian. 

measureCircuit.py generates a circuit which will measure a quantum state in the
correct bases to allow the energy to be calculated. This may require generating
multiple circuits if the same qubit needs to be measured in two perpendicular
bases (i.e. Z and X).

To find the minimum number of circuits needed to measure an entire Hamiltonian,
we treat the terms of H as nodes in a graph, G, where there are edges between
nodes indicate those two terms commute with one another. Finding the circuits
now becomes a clique finding problem which can be solved by the 
BronKerbosch algorithm.
"""


import time
import sys
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
import numpy as np
import pprint
import copy
import networkx as nx
from networkx.algorithms import approximation


class CommutativityType(object):
    def gen_comm_graph(term_array):
        raise NotImplementedError


class QWCCommutativity(CommutativityType):
    def gen_comm_graph(term_array):
        g = {}
        for i, term1 in enumerate(term_array):
            comm_array = []
            for j, term2 in enumerate(term_array):

                if i == j: continue
                commute = True
                for c1, c2 in zip(term1, term2):
                    if c1 == '*': continue
                    if (c1 != c2) and (c2 != '*'):
                        commute = False
                        break
                if commute:
                    comm_array += [''.join(term2)]
            g[''.join(term1)] = comm_array

        print('MEASURECIRCUIT: Generated graph for the Hamiltonian with {} nodes.'.format(len(g)))

        return g


class FullCommutativity(CommutativityType):
    def gen_comm_graph(term_array):
        g = {}

        for i, term1 in enumerate(term_array):
            comm_array = []
            for j, term2 in enumerate(term_array):

                if i == j: continue
                non_comm_indices = 0
                for c1, c2 in zip(term1, term2):
                    if c1 == '*': continue
                    if (c1 != c2) and (c2 != '*'):
                        non_comm_indices += 1
                if (non_comm_indices % 2) == 0:
                    comm_array += [''.join(term2)]
            g[''.join(term1)] = comm_array

        print('MEASURECIRCUIT: Generated graph for the Hamiltonian with {} nodes.'.format(len(g)))

        return g


def prune_graph(G,nodes):
    for n in nodes:
        neighbors = G.pop(n)
        for nn in neighbors:
            G[nn].remove(n)


def degeneracy_ordering(graph):
    """
    Produce a degeneracy ordering of the vertices in graph, as outlined in,
    Eppstein et. al. (arXiv:1006.5440)
    """

    # degen_order, will hold the vertex ordering
    degen_order = []
    
    while len(graph) > 0:
        # Populate D, an array containing a list of vertices of degree i at D[i]
        D = []
        for node in graph.keys():
            Dindex = len(graph[node])
            cur_len = len(D)
            if cur_len <= Dindex:
                while cur_len <= Dindex:
                    D.append([])
                    cur_len += 1
            D[Dindex].append(node)
        
        # Add the vertex with lowest degeneracy to degen_order
        for i in range(len(D)):
            if len(D[i]) != 0:
                v = D[i].pop(0)
                degen_order += [v]
                prune_graph(graph,[v])
                
    return degen_order


def degree_ordering(G):
    nodes = list(G.keys())
    return sorted(nodes, reverse=True, key=lambda n: len(G[n]))


def BronKerbosch_pivot(G,R,P,X,cliques):
    """
    For a given graph, G, find a maximal clique containing all of the vertices
    in R, some of the vertices in P, and none of the vertices in X.
    """
    if len(P) == 0 and len(X) == 0:
        # Termination case. If P and X are empty, R is a maximal clique
        cliques.append(R)
    else:
        # choose a pivot vertex
        pivot = next(iter(P.union(X)))
        # Recurse
        for v in P.difference(G[pivot]):
            # Recursion case. 
            BronKerbosch_pivot(G,R.union({v}),P.intersection(G[v]),
                               X.intersection(G[v]),cliques)
            P.remove(v)
            X.add(v)


def NetworkX_approximate_clique_cover(graph_dict):
    """
    NetworkX poly-time heuristic is based on
    Boppana, R., & Halldórsson, M. M. (1992).
    Approximating maximum independent sets by excluding subgraphs.
    BIT Numerical Mathematics, 32(2), 180–196. Springer.
    """
    G = nx.Graph()
    for src in graph_dict:
        for dst in graph_dict[src]:
            G.add_edge(src, dst)
    return approximation.clique_removal(G)[1]


def BronKerbosch(G):
    """
    Implementation of Bron-Kerbosch algorithm (Bron, Coen; Kerbosch, Joep (1973),
    "Algorithm 457: finding all cliques of an undirected graph", Commun. ACM,
    ACM, 16 (9): 575–577, doi:10.1145/362342.362367.) using a degree ordering
    of the vertices in G instead of a degeneracy ordering.
    See: https://en.wikipedia.org/wiki/Bron-Kerbosch_algorithm
    """

    max_cliques = []

    while len(G) > 0:
        P = set(G.keys())
        R = set()
        X = set()
        v = degree_ordering(G)[0]
        cliques = []
        BronKerbosch_pivot(G,R.union({v}),P.intersection(G[v]),
                           X.intersection(G[v]),cliques)

        #print('i = {}, current v = {}'.format(i,v))
        #print('# cliques: ',len(cliques))

        sorted_cliques = sorted(cliques, key=len, reverse=True)
        max_cliques += [sorted_cliques[0]]
        #print(sorted_cliques[0])

        prune_graph(G,sorted_cliques[0])

    return max_cliques


def generate_circuit_matrix(Nq, max_cliques):
    circuitMatrix = np.empty((Nq,len(max_cliques)),dtype=str)
    for i, clique in enumerate(max_cliques):
        # each clique will get its own circuit
        clique_list = list(clique)

        # Take the first string to be the circuit template, i.e. '****Z**Z'
        circStr = list(clique_list[0])
        #print(circStr)

        # Loop through the characters of the template and replace the '*'s
        # with a X,Y,Z found in another string in the same clique
        for j, char in enumerate(circStr):
            if char == '*':
                #print('j = {}, c = {}'.format(j,char))
                for tstr in clique_list[1:]:
                    # Search through the remaining strings in the clique
                    #print('tstr = {}, {} != * = {}'.format(tstr, tstr[j], (tstr[j] != '*')))
                    if tstr[j] != '*':
                        circStr[j] = tstr[j]
                        break
                
                if circStr[j] == '*':
                    # After searching through all of the strings in the clique
                    # the current char is still '*', this means none of the
                    # terms in this clique depend on this qubit -> measure in
                    # the Z basis.
                    circStr[j] = 'Z'

        # Once the circuit string is filled in, add it to the circuit matrix
        for q in range(Nq):
            circuitMatrix[q,i] = circStr[q]

    return circuitMatrix


def genMeasureCircuit(H, Nq, commutativity_type, clique_cover_method=BronKerbosch):
    """
    Take in a given Hamiltonian, H, and produce the minimum number of 
    necessary circuits to measure each term of H.

    Returns:
        List[QuantumCircuits]
    """

    start_time = time.time()

    term_reqs = np.full((len(H[1:]),Nq),'*',dtype=str)
    for i, term in enumerate(H[1:]):
        for op in term[1]:
            qubit_index = int(op[1:])
            basis = op[0]
            term_reqs[i][qubit_index] = basis

    # Generate a graph representing the commutativity of the Hamiltonian terms
    comm_graph = commutativity_type.gen_comm_graph(term_reqs)

    # Find a set of cliques within the graph where the nodes in each clique
    # are disjoint from one another.
    max_cliques = clique_cover_method(comm_graph)

    end_time = time.time()

    print('MEASURECIRCUIT: {} found {} unique circuits'.format(
        clique_cover_method.__name__, len(max_cliques)))
    et = end_time - start_time
    print('MEASURECIRCUIT: Elapsed time: {:.6f}s'.format(et))
    return max_cliques

    '''
    This section of the code produces a quantum circuit to measure each of
    the cliques found above. However, it can only handle QWC cliques since
    commuting groups like [XX,YY,ZZ] require a bit more handling.

    # Generate the circuitMatrix
    #   Number of rows = number of qubits
    #   Number of cols = number of circuits
    #   Each entry will have a value v <- {'Z','X','Y'} corresponding to the
    #   basis this qubit must be measured in.
    circuitMatrix = generate_circuit_matrix(Nq, max_cliques)

    # After the circuit matrix is computed, construct the quantum circuits
    circuitList = []
    for m in range(circuitMatrix.shape[1]):
        qr = QuantumRegister(Nq, name='qreg')
        cr = ClassicalRegister(Nq, name='creg')
        circ = QuantumCircuit(qr, cr)
        name = ''
        circ.barrier(qr)
        for n, basis in enumerate(circuitMatrix[:,m]):
            if basis == 'X':
                circ.h(qr[n])
            elif basis == 'Y':
                circ.sdg(qr[n])
                circ.h(qr[n])
            if basis == '*': basis = 'Z'
            name += basis
        circ.barrier(qr)
        for n in range(Nq):
            circ.measure(qr[n],cr[n])
        #print('name: ',name)
        circuitList += [(circ, name)]
        #circ.draw(scale=0.8, filename='throwaway/measure_{}_single_circ{}'.format(name,m), 
        #    output='mpl', plot_barriers=False, reverse_bits=True)

    return circuitList
    '''


def parseHamiltonian(myPath):
    H = []
    with open(myPath) as hFile:
        for i, line in enumerate(hFile):
            line = line.split()
            if i != 0:
                if "j" in line[0]:
                    print('Imaginary coefficient! -- skipping for now')
                    coef = 0.1
                else:
                    coef = float(line[0])
                ops = line[1:]
                H += [(coef, ops)]

    return H


# if __name__ == "__main__":
#     # change the number of qubits based on which hamiltonian is selected
#     hfile = 'hamiltonians/sampleH2.txt'
#     H = parseHamiltonian(hfile)

#     # Infer number of qubits from widest term in Hamiltonian
#     ops = [term[1] for term in H]
#     Nq = max([len(op) for op in ops])
#     print('%s qubits' % Nq)

#     for commutativity_type in [QWCCommutativity, FullCommutativity]:
#         cliques = genMeasureCircuit(H, Nq, commutativity_type)
#         for cliq in cliques:
#             print(cliq)
#         print()


# Setup of the problem

In [5]:
# Get ops and coeffs as list
split_input = input_hamiltonian.split()
coeffs = []
ops = []
for i, term in enumerate(split_input):
    if '0' in term:
        coeff = float(term)
        sign = 1 if split_input[i-1]=='+' else -1
        coeffs.append(sign * coeff)
    elif term[0] in ['I','X','Y','Z']:
        ops.append(term)

# Dictionary with coeffs
ops_dict = {}
for coeff, op in zip(coeffs, ops):
    ops_dict[op] = coeff

# Get ops in cirq format
cirq_ops = []
for op in ops:
    cirq_ops.append(get_openfermion_str(op))

# Get ops in cirq format, inverted
cirq_ops_inv = []
for op in ops:
    cirq_ops_inv.append(get_openfermion_str(op, reverse=True))

In [7]:
ops_dict

{'IIIYYIIIYY': 0.003034656830204855,
 'IIIXXIIIYY': 0.003034656830204855,
 'IIIYYIIIXX': 0.003034656830204855,
 'IIIXXIIIXX': 0.003034656830204855,
 'YZZZYIIIYY': -0.008373361424264817,
 'XZZZXIIIYY': -0.008373361424264817,
 'YZZZYIIIXX': -0.008373361424264817,
 'XZZZXIIIXX': -0.008373361424264817,
 'YZZYIIIIYY': 0.00211113766859809,
 'XZZXIIIIYY': 0.00211113766859809,
 'YZZYIIIIXX': 0.00211113766859809,
 'XZZXIIIIXX': 0.00211113766859809,
 'IIIIIIIIYY': -0.00491756976241806,
 'IIIIIIIIXX': -0.00491756976241806,
 'ZIIIIIIIYY': 0.010540187409026488,
 'ZIIIIIIIXX': 0.010540187409026488,
 'IZIIIIIIYY': -0.0011822832324725804,
 'IZIIIIIIXX': -0.0011822832324725804,
 'IIZIIIIIYY': -0.0011822832324725804,
 'IIZIIIIIXX': -0.0011822832324725804,
 'IIIZIIIIYY': -0.00154067008970742,
 'IIIZIIIIXX': -0.00154067008970742,
 'IIIIZIIIYY': 0.011733623912074194,
 'IIIIZIIIXX': 0.011733623912074194,
 'IIIIIZIIYY': 0.0027757462269049522,
 'IIIIIZIIXX': 0.0027757462269049522,
 'IIIIIIZIYY': 0.00362024875

# Openfermion Hamiltonian

We'll use `H_matrix_inv` as the exact `U` matrix. Note that we use the version with the inverted qubits because of the default qubit order Qiskit uses to convert circuits to unitaries. 

In [4]:
# Get Openfermion hamiltonian
H = q()
for coeff, op in zip(coeffs, cirq_ops):
    H += q(op, coeff)

H_matrix = get_sparse_operator(H)
U_matrix = ssl.expm(-1j * H_matrix).toarray()

# Get Openfermion hamiltonian, inverted
H_inv = q()
for coeff, op in zip(coeffs, cirq_ops_inv):
    H_inv += q(op, coeff)
    
H_matrix_inv = get_sparse_operator(H_inv)
U_matrix_inv = ssl.expm(-1j * H_matrix_inv).toarray()
U = lambda t: ssl.expm(-1j * t * H_matrix_inv).toarray()

It turns out we can discard the terms with very low coefficients. By just keeping the biggest 204 (discarding 72) we'll still get a total error below 0.1. 

In [10]:
#Using reduced coefficients
red_coeffs = []
red_ops = []
for coeff, op in zip(coeffs, ops):
    if abs(coeff) > 0.0035: #0.0035
        red_coeffs.append(coeff)
        red_ops.append(op)
        
print(len(coeffs), len(red_coeffs))

red_cirq_ops = []
for op in red_ops:
    red_cirq_ops.append(get_openfermion_str(op, reverse=True))
    
red_H = q()
for coeff, op in zip(red_coeffs, red_cirq_ops):
    red_H += q(op, coeff)
    
red_H_matrix = get_sparse_operator(red_H)
red_U_matrix = ssl.expm(-1j * red_H_matrix).toarray()

276 204


In [6]:
# Note: this doesn't account for Trotterization error, which is why it is much lower than 0.1
compare_with_exact(red_U_matrix, U_matrix_inv, global_phase=0.)

Error: 0.06291317001522562, Corr: 0.06291319186503104, Phase: -2.1860623608585703e-08.


## Reduced Hamiltonian

There are many ways of ordering the Pauli terms. We verified empirically that beginning with the Hamiltonian terms sorted in decreasing (absolute) value is a good starting point for subsequent changes.

In [11]:
# Terms associated with each coeff value
coeff_groups = {}
#for coeff, op in zip(coeffs, ops):
for coeff, op in zip(red_coeffs, red_ops):
    if coeff not in coeff_groups:
        coeff_groups[coeff] = [op]
    else:
        coeff_groups[coeff].append(op)
        
# Check whether each set of terms for a coeff value has full commutativity
coeff_commuting_groups = {}
for coeff, op_list in coeff_groups.items():
    com = True
    for op1 in op_list:
        for op2 in op_list:
            com = com and ops_commute(op1, op2)
    coeff_commuting_groups[coeff] = com

In [12]:
coeff_groups_sorted = dict(sorted(coeff_groups.items(), key = lambda x: -abs(x[0])))

In [13]:
coeff_groups_sorted

{1.0709274663656798: ['IIIIIIIIII'],
 -0.5772920990654372: ['IIIIIZIIII'],
 -0.5772920990654371: ['ZIIIIIIIII'],
 -0.4244817531727134: ['IIZIIIIIII'],
 -0.42448175317271336: ['IIIIIIZIII', 'IIIIIIIZII'],
 -0.4244817531727133: ['IZIIIIIIII'],
 -0.3899177647415215: ['IIIZIIIIII'],
 -0.38991776474152134: ['IIIIIIIIZI'],
 -0.30101532158947975: ['IIIIZIIIII', 'IIIIIIIIIZ'],
 0.12357087224898464: ['IIIIZIIIIZ'],
 0.11433954684977561: ['IIIIZZIIII', 'ZIIIIIIIIZ'],
 0.11409163501020725: ['ZIIIIZIIII'],
 0.08470391802239534: ['IIIZIIIIZI'],
 0.08360121967246183: ['ZIIIZIIIII', 'IIIIIZIIIZ'],
 0.0782363777898523: ['IZIIIIZIII', 'IIZIIIIZII'],
 0.07055432072184756: ['IIIZIIZIII', 'IIIZIIIZII', 'IZIIIIIIZI', 'IIZIIIIIZI'],
 0.06980180803300681: ['IIZIIIZIII', 'IZIIIIIZII'],
 0.06878552428444665: ['IIIIZIZIII', 'IIIIZIIZII', 'IZIIIIIIIZ', 'IIZIIIIIIZ'],
 0.06732342777645686: ['IZIIIZIIII', 'IIZIIZIIII', 'ZIIIIIZIII', 'ZIIIIIIZII'],
 0.06558452315458405: ['IZZIIIIIII', 'IIIIIIZZII'],
 0.062788763434

In [14]:
sorted_ops = []
for coeff, op_list in coeff_groups_sorted.items():
    for op in op_list:
        sorted_ops.append(op)

In [11]:
red_ops_no_I = deepcopy(red_ops)
red_ops_no_I.remove('IIIIIIIIII')

In [12]:
ops2 = red_ops_no_I

comm_list = []
for op_1, op_2 in zip(ops2[:-1],ops2[1:]):
    comm_list.append(ops_commute(op_1, op_2))

In [13]:
from random import gauss

In [ ]:
new_pos = sorted(list(range(len(red_ops_no_I))), key=lambda i: gauss(i * 1., 1))
thermal_red_ops_no_I = [red_ops_no_I[i] for i in new_pos]

In [ ]:
circ = QuantumCircuit(10)
U_test = SummedOp([PauliOp(Pauli(op), coeff=ops_dict[op]) for op in sorted_ops]).exp_i()
circ += PauliTrotterEvolution().convert(U_test).to_circuit()

In [ ]:
for _ in range(5):
    circ = transpile(circ, optimization_level=3, approximation_degree=0.0)
    circ = circ.decompose()
    print(circ.depth(), circ.count_ops())

In [ ]:
trot_U_matrix = Operator(circ).data

In [ ]:
# 204 terms
compare_with_exact(trot_U_matrix, U_matrix_inv)

In [ ]:
circ.draw()

In [ ]:
qc = transpile(qc, optimization_level=3, approximation_degree=1.0)

In [ ]:
print(qc.depth(), qc.count_ops())

In [ ]:
trot_U_matrix = Operator(qc).data

In [ ]:
compare_with_exact(trot_U_matrix, U_matrix_inv)

This result is exactly the same as straightforward Lie-Trotter application, which indicates that PauliTrotterEvolution is not reordering the Hamiltonian terms.

# Following arxiv 2001.05983

We initially tested all methods proposed in <https://arxiv.org/abs/2001.05983>. However, it turns out the original input Hamiltonian and the sorted Hamiltonian, when implemented with Lie-Trotter (1 time step) have errors (0.09) considerably below a random order (0.3) or the solutions resulting from the paper's proposed method (0.2). Therefore, we used some of the paper's ideas, but significantly built upon them, to account for the different restrictions in our setup. 

**Warning!** To simplify the analysis, here we discard the I term, as it just adds a global phase.

In [15]:
sorted_ops

['IIIIIIIIII',
 'IIIIIZIIII',
 'ZIIIIIIIII',
 'IIZIIIIIII',
 'IIIIIIZIII',
 'IIIIIIIZII',
 'IZIIIIIIII',
 'IIIZIIIIII',
 'IIIIIIIIZI',
 'IIIIZIIIII',
 'IIIIIIIIIZ',
 'IIIIZIIIIZ',
 'IIIIZZIIII',
 'ZIIIIIIIIZ',
 'ZIIIIZIIII',
 'IIIZIIIIZI',
 'ZIIIZIIIII',
 'IIIIIZIIIZ',
 'IZIIIIZIII',
 'IIZIIIIZII',
 'IIIZIIZIII',
 'IIIZIIIZII',
 'IZIIIIIIZI',
 'IIZIIIIIZI',
 'IIZIIIZIII',
 'IZIIIIIZII',
 'IIIIZIZIII',
 'IIIIZIIZII',
 'IZIIIIIIIZ',
 'IIZIIIIIIZ',
 'IZIIIZIIII',
 'IIZIIZIIII',
 'ZIIIIIZIII',
 'ZIIIIIIZII',
 'IZZIIIIIII',
 'IIIIIIZZII',
 'IZIIZIIIII',
 'IIZIZIIIII',
 'IIIIIIZIIZ',
 'IIIIIIIZIZ',
 'ZZIIIIIIII',
 'ZIZIIIIIII',
 'IIIIIZZIII',
 'IIIIIZIZII',
 'IIIZIZIIII',
 'ZIIIIIIIZI',
 'IZIZIIIIII',
 'IIZZIIIIII',
 'IIIIIIZIZI',
 'IIIIIIIZZI',
 'IIIIZIIIZI',
 'IIIZIIIIIZ',
 'ZIIZIIIIII',
 'IIIIIZIIZI',
 'IIIZZIIIII',
 'IIIIIIIIZZ',
 'ZIIIIYZZZY',
 'ZIIIIXZZZX',
 'YZZZYZIIII',
 'XZZZXZIIII',
 'IIIIZYZZZY',
 'IIIIZXZZZX',
 'YZZZYIIIIZ',
 'XZZZXIIIIZ',
 'YZZZYYZZZY',
 'XZZZXYZZZY',
 'YZZZYXZZ

In [16]:
ops_no_I = deepcopy(sorted_ops)
ops_no_I.remove('IIIIIIIIII')

ops2 = ops_no_I

In [17]:
ops2

['IIIIIZIIII',
 'ZIIIIIIIII',
 'IIZIIIIIII',
 'IIIIIIZIII',
 'IIIIIIIZII',
 'IZIIIIIIII',
 'IIIZIIIIII',
 'IIIIIIIIZI',
 'IIIIZIIIII',
 'IIIIIIIIIZ',
 'IIIIZIIIIZ',
 'IIIIZZIIII',
 'ZIIIIIIIIZ',
 'ZIIIIZIIII',
 'IIIZIIIIZI',
 'ZIIIZIIIII',
 'IIIIIZIIIZ',
 'IZIIIIZIII',
 'IIZIIIIZII',
 'IIIZIIZIII',
 'IIIZIIIZII',
 'IZIIIIIIZI',
 'IIZIIIIIZI',
 'IIZIIIZIII',
 'IZIIIIIZII',
 'IIIIZIZIII',
 'IIIIZIIZII',
 'IZIIIIIIIZ',
 'IIZIIIIIIZ',
 'IZIIIZIIII',
 'IIZIIZIIII',
 'ZIIIIIZIII',
 'ZIIIIIIZII',
 'IZZIIIIIII',
 'IIIIIIZZII',
 'IZIIZIIIII',
 'IIZIZIIIII',
 'IIIIIIZIIZ',
 'IIIIIIIZIZ',
 'ZZIIIIIIII',
 'ZIZIIIIIII',
 'IIIIIZZIII',
 'IIIIIZIZII',
 'IIIZIZIIII',
 'ZIIIIIIIZI',
 'IZIZIIIIII',
 'IIZZIIIIII',
 'IIIIIIZIZI',
 'IIIIIIIZZI',
 'IIIIZIIIZI',
 'IIIZIIIIIZ',
 'ZIIZIIIIII',
 'IIIIIZIIZI',
 'IIIZZIIIII',
 'IIIIIIIIZZ',
 'ZIIIIYZZZY',
 'ZIIIIXZZZX',
 'YZZZYZIIII',
 'XZZZXZIIII',
 'IIIIZYZZZY',
 'IIIIZXZZZX',
 'YZZZYIIIIZ',
 'XZZZXIIIIZ',
 'YZZZYYZZZY',
 'XZZZXYZZZY',
 'YZZZYXZZZX',
 'XZZZXXZZ

## Helper functions

As these are at the heart of our implementation, we present them here instead of in `aux_functions.py`.

In [28]:
def final_order(min_clique_cover):
    """
    Takes a list of lists of commuting Pauli terms,
    and uses several procedures to minimize the depth
    count of the circuit.
    
    Parameters
    ----------
    min_clique_cover : list
        List of lists of mutually commuting terms.
        
    Returns
    -------
    list
        List of tuples `(op, qb)`, with `op` the Pauli
    term, and `qb` the qubit which should be used for 
    the `Rz` gate.
    """

    # For each set of commuting terms, rearrange 
    # the terms to parallelize evolutions that don't 
    # share qubits.
    # It does this by solving a minimum-clique-cover
    # problem, using the Bron-Kerbosh algorithm.
    min_overlap_cover = []
    for clique in min_clique_cover:
        overlap_dict = {}
        for op_1 in clique:
            comm_list = []
            for op_2 in clique:
                if op_1 == op_2:
                    continue   
                if ops_do_not_overlap(op_1, op_2):
                    comm_list.append(op_2)
            overlap_dict[op_1] = comm_list

        min_cover = BronKerbosch(overlap_dict)
        min_cover = [sorted(list(ov_clique), reverse=True) for ov_clique in min_cover]
        min_overlap_cover.append(min_cover)

    # Merge the parallel evolutions (temporarily),
    # and add a unique identifier to each term,
    # as some merged terms may be identical.
    min_target_cond = []
    for i, clique in enumerate(min_overlap_cover):
        min_target_cond.append([])
        for j, overlap in enumerate(clique):
            pauli = ['I' for _ in range(10)]
            for op in overlap:
                for p, pop in enumerate(list(op)):
                    if pop!='I':
                        pauli[p] = pop
            pauli = ''.join(pauli)
            min_target_cond[i].append((pauli, j))

    
    # Build the edges of a graph with the merged
    # terms as nodes, and the edge weights as the
    # similarity between the terms.
    # Solve a traveling-salesman-problem to find
    # the term order that minimizes Pauli gate 
    # differences between neighboring terms.
    # This part is inspired by the TSP portion of
    # 2001.05983, but the weight function is more
    # heuristic, as the paper's assumptions are 
    # not fully satisfied here.
    min_target_cond_ord = []
    for clique in min_target_cond:
        ops3 = clique
        if len(clique)==1:
            min_target_cond_ord.append(clique)
            continue
        edges = []
        for i, (op_1, label_1) in enumerate(ops3):
            for j, (op_2, label_2) in enumerate(ops3):
                if i == j:
                    continue
                edges.append(((op_1, label_1), (op_2, label_2), match_weight(op_1, op_2)))

        #print(ops3, edges)
        G = nx.Graph()
        G.add_nodes_from(ops3)
        G.add_weighted_edges_from(edges)

        min_target_cond_ord.append(tsp(G, cycle=False))

   
    # Unmerge the merged terms.
    min_target_cover_ord = []
    for i, clique in enumerate(min_target_cond_ord):
        min_target_cover_ord.append([])
        for op in clique:
            min_target_cover_ord[i].append(min_overlap_cover[i][op[1]])

    # We wish to choose, for each term, the qubit 
    # that is going to have the Rz gate in such a way
    # that the number of changes of that qubit from one
    # term to the next is minimized.
    # This allows us to maximize the CNOT cancellation 
    # potential.
    # This problem can be construed as a Set Cover problem;
    # here that problem is solved by using a greedy algorithm.
    # For terms that are run in parallel, we consider the merged
    # term. This means that, after unmerging, some terms will 
    # suboptimal qubit choices for the Rz gate. Nonetheless,
    # since our Hamiltonian is not highly parallelizable, we
    # expect this problem to have a negligible impact.
    target_list = [item for items in min_target_cond_ord for item in items]
    covering_sets = []
    set_names = []
    tmp_list = [list(op) for op, pos in target_list]
    for qb in range(10):
        #covering_sets.append([])
        qb_list = []
        qb_count = 0
        for i, op in enumerate(tmp_list):
            if op[qb] != 'I':
                qb_list.append(i)
            elif len(qb_list)!=0:
                covering_sets.append(qb_list)
                set_names.append((9-qb, qb_count))
                qb_count += 1
                qb_list = []
        if len(qb_list)!=0:
            covering_sets.append(qb_list)
            set_names.append((9-qb, qb_count))
            qb_count += 1
            qb_list = []

    # Set cover problem
    to_cover = set(range(len(target_list)))
    sets_available = [(i, set(row)) for i, row in enumerate(covering_sets)]
    sets_to_use = []
    while len(to_cover) > 0:
        max_val = -1
        argmax = -1
        for ind, (i, row) in enumerate(sets_available):
            val = len(to_cover & row)
            if val > max_val:
                max_val = val
                argmax = i
                argind = ind

        row_max = sets_available.pop(argind)
        sets_to_use.append(argmax)
        to_cover = to_cover ^ (to_cover & row_max[1])

    covering_sets_to_use = [covering_sets[i] for i in sets_to_use]
    set_names_to_use = [set_names[i] for i in sets_to_use]

    target_dict = {}
    for i, row in enumerate(covering_sets_to_use):
        for ind in row:
            target_dict[ind] = set_names_to_use[i][0]

    flat_cover_ord = [item for items in min_target_cover_ord for item in items]

    ops_w_targets = []
    for ind, col in enumerate(flat_cover_ord):
        ops_w_targets.append([])
        for op in col:
            ops_w_targets[-1].append((op, target_dict[ind]))

    # We finally flatten the list, to implement the circuit.
    flat_list = [item for items in ops_w_targets for item in items]

    return flat_list

In [16]:
def pauli_circuit(op, coeff, cnot='star', t=1.):
    """
    Implements the evolution of a Pauli term.
    
    Parameters
    ----------
    op : str, tuple
        The Pauli term to be implement (in Qiskit format). If
    tuple, the first entry contains the Pauli term, and the 
    second contains the qubit index where the Rz rotation should
    be applied.
    coeff : float
        The associated coefficient.
    cnot : {'star','ladder','parallel'}
        How to implement the CNOTs. The `star` and `ladder`
    methods are those known in the literature. The `parallel`
    method implements the `n` CNOTs using `log_2(n)` depth.
    t : float
        Evolution time.
        
    Returns
    -------
    QuantumCircuit
        Quantum circuit.
        
    Notes
    -----
    The `parallel` method results in lower depth circuits at the 
    outset, but the pyZX and transpile optimizations reduce the 
    `star` method more extensively, so it results in the circuits 
    with a lower depth than `parallel` by about 1%, on average.
    The `ladder` method is not competitive, depth-wise.
    
    Warnings
    --------
    The `parallel` method has a unknown bug, which occasionally
    causes to wrongly implement the CNOTs.
    """
    
    if isinstance(op, tuple):
        target_qb = op[1]
        op = op[0]
    else:
        target_qb = -1
    
    qc = QuantumCircuit(10)
    qc_cx = QuantumCircuit(10)
    non_I_qbs = []
    
    # Basis change
    for i, pauli in enumerate(reversed(list(op))):
        if pauli == 'Z':
            non_I_qbs.append(i)
        elif pauli == 'X':
            qc_cx.h(i)
            non_I_qbs.append(i)
        elif pauli == 'Y':
            qc_cx.sdg(i)
            qc_cx.h(i)
            non_I_qbs.append(i)
    
    # Identity case
    if len(non_I_qbs) == 0:
        return qc
    
    # CNOT gates
    if target_qb == -1:
        control_qbs = list(reversed(non_I_qbs[1:]))
        target = non_I_qbs[0]
    else:
        try:
            pos = non_I_qbs.index(target_qb)
        except ValueError:
            pos = 0
        control_qbs = list(reversed(non_I_qbs[:pos]+non_I_qbs[pos+1:]))
        target = non_I_qbs[pos]
    
    if cnot=='star':
        for qb in control_qbs:
            qc_cx.cx(qb, target)
    elif cnot=='ladder':
        for i in range(len(control_qbs)):
            if i == len(control_qbs)-1:
                qc_cx.cx(control_qbs[i], target)
            else:
                qc_cx.cx(control_qbs[i], control_qbs[i+1])
    elif cnot=='parallel':
        remaining_qbs = list(reversed(non_I_qbs))
        remaining_qbs.remove(target)
        remaining_qbs += [target]
        while len(remaining_qbs) > 1:
            targets = remaining_qbs[1::2]
            controls = remaining_qbs[0::2]
            #print(remaining_qbs, controls, targets)
            for i in range(len(targets)):
                qc_cx.cx(controls[i], targets[i])
            remaining_qbs = targets + ([controls[-1]] if len(remaining_qbs)%2==1 else [])
        
    qc += qc_cx
    qc.rz(2*coeff*t, target)
    qc += qc_cx.inverse()
            
    return qc

# Optimal order
## Finding commuting groups

In [21]:
# We start by implementing the first step of 2001.05983,
# which solves a minimum-clique-cover problem to find
# a term grouping such that each group's terms mutually
# commute, and the number of terms is minimized.
# This is solved by using the Bron-Kerbosh algorithm.

# Putting this in the format expected by vqe_term_commuting.py
comm_dict = {}
for op_1 in ops2:
    comm_list = []
    for op_2 in ops2:
        if op_1 == op_2:
            continue   
        if ops_commute(op_1, op_2):
            comm_list.append(op_2)
    comm_dict[op_1] = comm_list

min_clique_cover_orig = BronKerbosch(comm_dict)

In [25]:
# The result varies with the run, it may be as low as 9. Paradoxically,
# a lower number of groups actually seems to lead to worse results.
# Here, we present a relatively high-number grouping, with 14 groups.
min_clique_cover_orig[1]

{'IIIIIIIIXX',
 'IIIIIIIZXX',
 'IIIIIIZIXX',
 'IIIIIXIZXI',
 'IIIIIXZIXI',
 'IIIIIXZZIX',
 'IIIIIXZZXI',
 'IIIIIYZZYZ',
 'IIIIIYZZZY',
 'IIIYYIIIII',
 'IIIYYYZZZY',
 'IIZYYIIIII',
 'IZIYYIIIII',
 'XZZXZIIIII',
 'XZZZXIIIII',
 'XZZZXIIIXX',
 'XZZZXXZZXI',
 'XZZZXYZZZY',
 'YIZYIIIIII',
 'YZIYIIIIII',
 'YZZIYIIIII',
 'YZZYIIIIII',
 'YZZYIXZZXI',
 'YZZYIYZZZY'}

In [26]:
# We reorder the terms within each group, we try to keep the performance
# near the original sorted order, which we already low has low error.

min_clique_cover = []
for clique in min_clique_cover_orig:
    lclique = list(clique)
    min_clique_cover.append(lclique)

min_clique_cover = [sorted(clique, key=lambda op: abs(ops_dict[op]), reverse=True) for clique in min_clique_cover]

In [29]:
# Get optimal ordering
flat_list = final_order(min_clique_cover)

In [ ]:
flat_list

In [72]:
# Implement circuit
circ = QuantumCircuit(10)
for op in flat_list:
    circ += pauli_circuit(op, ops_dict[op[0]], cnot='star', t=1.)

In [73]:
# Original circuit depth, without optimizations.
# The `parallel` variant would lead to better results here,
# when compared with `star`.
print(circ.depth(), circ.count_ops())

1621 OrderedDict([('cx', 1402), ('h', 928), ('sdg', 232), ('s', 232), ('rz', 203)])


In [74]:
# Optimize using pyzx
zx_circ = zx.Circuit.from_qasm(circ.qasm())
g = zx_circ.to_graph()
zx.full_reduce(g, quiet=True)
zx_opt = zx.extract_circuit(g.copy())
circ = QuantumCircuit.from_qasm_str(zx_opt.to_qasm())

In [75]:
# Result with pyzx. It uses cz gates, so it is not acceptable.
print(circ.depth(), circ.count_ops())

736 OrderedDict([('cz', 414), ('h', 372), ('cx', 289), ('rz', 208)])


In [76]:
# Iteratively optimize the circuit using Qiskit.
for _ in range(5):
    circ = transpile(circ, optimization_level=3, approximation_degree=1.0)
    circ = circ.decompose()
    print(circ.depth(), circ.count_ops())

1067 OrderedDict([('h', 828), ('cx', 703), ('u2', 370), ('u1', 208)])
892 OrderedDict([('cx', 703), ('u3', 578), ('u2', 412)])
892 OrderedDict([('cx', 703), ('u', 578), ('u3', 412)])
892 OrderedDict([('u', 990), ('cx', 703)])
892 OrderedDict([('u', 990), ('cx', 703)])


In [77]:
# Convert the final circuit to a unitary
trot_U_matrix = Operator(circ).data

In [78]:
# Compare with exact case, verifying that the error is still within 0.1
# Note that some choices of commuting groups will result with circuits
# with errors above the threshold. Empirically, results that have 13 or 14
# commuting groups seem to be good enough.
compare_with_exact(trot_U_matrix, U_matrix_inv)

Error: 0.09839344812908042, Corr: 0.09839306108065178, Phase: 3.875176689759584e-07.


In [46]:
# Save QASM circuit and verify that it still has the original depth.
circ_qasm = circ.qasm()
new_circ = QuantumCircuit.from_qasm_str(circ_qasm)
print(new_circ.depth(), new_circ.count_ops())

892 OrderedDict([('u', 990), ('cx', 703)])


In [61]:
print(circ_qasm)

OPENQASM 2.0;
include "qelib1.inc";
qreg q[10];
cx q[6],q[7];
cx q[7],q[6];
cx q[6],q[7];
cx q[3],q[7];
cx q[5],q[6];
cx q[6],q[5];
cx q[5],q[6];
cx q[1],q[5];
cx q[2],q[6];
cx q[5],q[1];
cx q[1],q[5];
cx q[0],q[5];
u(pi/2,0,pi) q[1];
u(pi/2,0,pi) q[1];
cx q[5],q[0];
cx q[0],q[5];
cx q[0],q[1];
u(pi/2,0,pi) q[1];
u(pi/2,0,pi) q[1];
u(pi/2,0,pi) q[5];
u(pi/2,0,pi) q[5];
cx q[6],q[2];
cx q[2],q[6];
u(pi/2,0,pi) q[2];
u(pi/2,0,pi) q[6];
u(pi/2,0,pi) q[6];
cx q[7],q[3];
cx q[3],q[7];
u(pi/2,0,pi) q[3];
u(pi/2,0,pi) q[7];
u(pi/2,0,pi) q[7];
u(pi/2,0,pi) q[8];
cx q[4],q[9];
cx q[9],q[4];
cx q[4],q[9];
u(pi/2,0,pi) q[4];
u(pi/2,0,pi) q[4];
cx q[1],q[4];
cx q[0],q[4];
u(pi/2,0,pi) q[4];
u(pi/2,0,pi) q[4];
u(pi/2,0,pi) q[9];
u(pi/2,0,pi) q[9];
cx q[4],q[9];
cx q[1],q[9];
cx q[0],q[9];
cx q[0],q[6];
u(pi/2,0,pi) q[4];
u(pi/2,0,pi) q[6];
u(pi/2,0,pi) q[9];
cx q[3],q[9];
cx q[3],q[4];
u(pi/2,0,pi) q[3];
cx q[1],q[3];
cx q[0],q[3];
cx q[0],q[1];
u(pi/2,0,pi) q[3];
u(pi/2,0,pi) q[3];
u(pi/2,0,pi) q[

In [86]:
# Convert the final circuit to a unitary
trot_U_matrix = Operator(new_circ).data

In [87]:
# QASM doesn't store the global phase of the circuit, so we need to apply it term to get the correct error.
compare_with_exact(trot_U_matrix, U_matrix_inv, circuit_phase=circ.global_phase + np.pi/2)

Error: 0.09839363517169733, Corr: 0.09839307173976077, Phase: 5.641150174879282e-07.
